<a href="https://colab.research.google.com/github/Mickymick23/GEOG5003M/blob/main/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hi